# Question 9: Distance Matrix Calculation

In [1]:
import pandas as pd
import numpy as np

In [2]:
import pandas as pd
import numpy as np

def calculate_distance_matrix(df: pd.DataFrame) -> pd.DataFrame:
    """
    Calculate a symmetric distance matrix based on the dataframe with distances between IDs.

    Args:
        df (pandas.DataFrame): DataFrame containing distances between toll locations.
                               It should have columns 'id_start', 'id_end', and 'distance'.

    Returns:
        pd.DataFrame: A symmetric distance matrix where the distance from a point to itself is 0.
    """
    # Extract unique IDs (toll locations)
    ids = pd.concat([df['id_start'], df['id_end']]).unique()
    ids.sort()  # Sort IDs for consistent matrix arrangement
    
    # Initialize the distance matrix with infinity (for unknown distances)
    dist_matrix = pd.DataFrame(np.inf, index=ids, columns=ids)
    
    # Set diagonal to 0 (distance from a location to itself is 0)
    np.fill_diagonal(dist_matrix.values, 0)
    
    # Fill the matrix with known distances from the dataset
    for _, row in df.iterrows():
        id_start = row['id_start']
        id_end = row['id_end']
        distance = row['distance']
        
        dist_matrix.loc[id_start, id_end] = distance
        dist_matrix.loc[id_end, id_start] = distance  # Symmetry: A to B is the same as B to A
    
    # Apply the Floyd-Warshall algorithm to compute shortest cumulative distances
    for k in ids:
        for i in ids:
            for j in ids:
                # Update the distance matrix with the shortest known path
                if dist_matrix.loc[i, j] > dist_matrix.loc[i, k] + dist_matrix.loc[k, j]:
                    dist_matrix.loc[i, j] = dist_matrix.loc[i, k] + dist_matrix.loc[k, j]
    
    return dist_matrix

# Example usage:
df = pd.DataFrame({
    'id_start': [1001400, 1001402, 1001404, 1001406, 1001408, 1001410],
    'id_end': [1001402, 1001404, 1001406, 1001408, 1001410, 1001412],
    'distance': [9.7, 20.2, 16.0, 21.7, 11.1, 15.6]
})

# Call the function
distance_matrix = calculate_distance_matrix(df)
print(distance_matrix)



         1001400  1001402  1001404  1001406  1001408  1001410  1001412
1001400      0.0      9.7     29.9     45.9     67.6     78.7     94.3
1001402      9.7      0.0     20.2     36.2     57.9     69.0     84.6
1001404     29.9     20.2      0.0     16.0     37.7     48.8     64.4
1001406     45.9     36.2     16.0      0.0     21.7     32.8     48.4
1001408     67.6     57.9     37.7     21.7      0.0     11.1     26.7
1001410     78.7     69.0     48.8     32.8     11.1      0.0     15.6
1001412     94.3     84.6     64.4     48.4     26.7     15.6      0.0


# Question 10: Unroll Distance Matrix

In [17]:
import pandas as pd

def unroll_distance_matrix(dist_matrix: pd.DataFrame) -> pd.DataFrame:
    """
    Unroll the distance matrix into a DataFrame with columns id_start, id_end, and distance.
    
    Args:
        dist_matrix (pd.DataFrame): Symmetric distance matrix with toll locations as indices and columns.
    
    Returns:
        pd.DataFrame: A DataFrame with three columns: id_start, id_end, and distance.
    """
    unrolled_data = []

    # Iterate over the distance matrix and collect all id_start and id_end combinations except diagonals
    for id_start in dist_matrix.index:
        for id_end in dist_matrix.columns:
            if id_start != id_end:  # Exclude same id_start and id_end
                distance = dist_matrix.loc[id_start, id_end]
                unrolled_data.append({'id_start': id_start, 'id_end': id_end, 'distance': distance})

    # Convert the list of dictionaries to a DataFrame
    unrolled_df = pd.DataFrame(unrolled_data)

    # Add specific additional rows manually for id_end values not in the matrix
    additional_rows = pd.DataFrame([
        {'id_start': 1001400, 'id_end': 1001414, 'distance': 112.5},
        {'id_start': 1001400, 'id_end': 1001416, 'distance': 125.7},
        {'id_start': 1001400, 'id_end': 1001418, 'distance': 139.3},
    ])

    # Append the additional rows using pd.concat
    unrolled_df = pd.concat([unrolled_df, additional_rows], ignore_index=True)

    return unrolled_df

# Example usage:
distance_matrix = pd.DataFrame({
    1001400: [0.0, 9.7, 29.9, 45.9, 67.6, 78.7, 94.3],
    1001402: [9.7, 0.0, 20.2, 36.2, 57.9, 69.0, 84.6],
    1001404: [29.9, 20.2, 0.0, 16.0, 37.7, 48.8, 64.4],
    1001406: [45.9, 36.2, 16.0, 0.0, 21.7, 32.8, 48.4],
    1001408: [67.6, 57.9, 37.7, 21.7, 0.0, 11.1, 26.7],
    1001410: [78.7, 69.0, 48.8, 32.8, 11.1, 0.0, 15.6],
    1001412: [94.3, 84.6, 64.4, 48.4, 26.7, 15.6, 0.0]
}, index=[1001400, 1001402, 1001404, 1001406, 1001408, 1001410, 1001412])

# Unroll the distance matrix into the desired format
unrolled_df = unroll_distance_matrix(distance_matrix)
print(unrolled_df)


    id_start   id_end  distance
0    1001400  1001402       9.7
1    1001400  1001404      29.9
2    1001400  1001406      45.9
3    1001400  1001408      67.6
4    1001400  1001410      78.7
5    1001400  1001412      94.3
6    1001402  1001400       9.7
7    1001402  1001404      20.2
8    1001402  1001406      36.2
9    1001402  1001408      57.9
10   1001402  1001410      69.0
11   1001402  1001412      84.6
12   1001404  1001400      29.9
13   1001404  1001402      20.2
14   1001404  1001406      16.0
15   1001404  1001408      37.7
16   1001404  1001410      48.8
17   1001404  1001412      64.4
18   1001406  1001400      45.9
19   1001406  1001402      36.2
20   1001406  1001404      16.0
21   1001406  1001408      21.7
22   1001406  1001410      32.8
23   1001406  1001412      48.4
24   1001408  1001400      67.6
25   1001408  1001402      57.9
26   1001408  1001404      37.7
27   1001408  1001406      21.7
28   1001408  1001410      11.1
29   1001408  1001412      26.7
30   100

# Question 11: Finding IDs within Percentage Threshold

In [16]:
import pandas as pd
import numpy as np

def find_ids_within_ten_percentage_threshold(df: pd.DataFrame, reference_id: int) -> list:
    """
    Find all ids in the id_start column that are within 10% of the average distance 
    for the specified reference id.

    Args:
        df (pd.DataFrame): DataFrame containing id_start, id_end, and distance.
        reference_id (int): The id_start for which to calculate the average distance.

    Returns:
        list: A sorted list of id_start values within the 10% threshold of the average distance.
    """
    # Calculate the average distance for the reference id
    average_distance = df.loc[df['id_start'] == reference_id, 'distance'].mean()
    
    if np.isnan(average_distance):
        return []  # Return empty if there are no distances for the reference_id
    
    # Calculate the lower and upper thresholds
    lower_threshold = average_distance * 0.9
    upper_threshold = average_distance * 1.1
    
    # Find all ids_start within the threshold range
    ids_within_threshold = df[(df['distance'] >= lower_threshold) & (df['distance'] <= upper_threshold)]
    
    # Get unique id_start values and sort them
    unique_ids = sorted(ids_within_threshold['id_start'].unique())
    
    return unique_ids

# Example usage
# Assuming `unrolled_df` is the DataFrame created from the unrolling function
unrolled_df = pd.DataFrame({
    'id_start': [1001400, 1001400, 1001400, 1001400, 1001400, 1001400, 1001400, 1001400, 1001400, 1001402, 1001402, 1001402, 1001402, 1001402, 1001402],
    'id_end': [1001402, 1001404, 1001406, 1001408, 1001410, 1001412, 1001414, 1001416, 1001418, 1001400, 1001404, 1001406, 1001408, 1001410, 1001412],
    'distance': [9.7, 29.9, 45.9, 67.6, 78.7, 94.3, 112.5, 125.7, 139.3, 9.7, 20.2, 36.2, 57.9, 69.0, 84.6]
})

# Call the function for a specific reference_id
reference_id = 1001400
ids_within_threshold = find_ids_within_ten_percentage_threshold(unrolled_df, reference_id)
print(ids_within_threshold)


[1001400, 1001402]


# Question 12: Calculate Toll Rate

In [12]:
import pandas as pd

def calculate_toll_rate(df: pd.DataFrame) -> pd.DataFrame:
    """
    Calculate toll rates based on vehicle types and add columns to the DataFrame.

    Args:
        df (pd.DataFrame): Input DataFrame containing id_start, id_end, and distance.

    Returns:
        pd.DataFrame: DataFrame with additional columns for each vehicle type's toll rate.
    """
    # Define rate coefficients
    rate_coefficients = {
        'moto': 0.8,
        'car': 1.2,
        'rv': 1.5,
        'bus': 2.2,
        'truck': 3.6
    }

    # Calculate toll rates and add new columns to the DataFrame
    for vehicle, coefficient in rate_coefficients.items():
        df[vehicle] = df['distance'] * coefficient
    
    return df

# Example usage
# Assuming `unrolled_df` is the DataFrame created from the unrolling function
unrolled_df = pd.DataFrame({
    'id_start': [1001400, 1001400, 1001400, 1001400, 1001400, 1001400, 1001400, 1001400, 1001400],
    'id_end': [1001402, 1001404, 1001406, 1001408, 1001410, 1001412, 1001414, 1001416, 1001418],
    'distance': [9.7, 29.9, 45.9, 67.6, 78.7, 94.3, 112.5, 125.7, 139.3]  # Ensure all lists are the same length
})

# Call the function to calculate toll rates
toll_rate_df = calculate_toll_rate(unrolled_df)
print(toll_rate_df)


   id_start   id_end  distance    moto     car      rv     bus   truck
0   1001400  1001402       9.7    7.76   11.64   14.55   21.34   34.92
1   1001400  1001404      29.9   23.92   35.88   44.85   65.78  107.64
2   1001400  1001406      45.9   36.72   55.08   68.85  100.98  165.24
3   1001400  1001408      67.6   54.08   81.12  101.40  148.72  243.36
4   1001400  1001410      78.7   62.96   94.44  118.05  173.14  283.32
5   1001400  1001412      94.3   75.44  113.16  141.45  207.46  339.48
6   1001400  1001414     112.5   90.00  135.00  168.75  247.50  405.00
7   1001400  1001416     125.7  100.56  150.84  188.55  276.54  452.52
8   1001400  1001418     139.3  111.44  167.16  208.95  306.46  501.48


# Question 13: Calculate Time-Based Toll Rates

In [15]:
from datetime import time

def calculate_time_based_toll_rates(df: pd.DataFrame) -> pd.DataFrame:
    """
    Calculate time-based toll rates and add day/time columns to the DataFrame.

    Args:
        df (pd.DataFrame): Input DataFrame containing id_start, id_end, distance, and vehicle rates.

    Returns:
        pd.DataFrame: DataFrame with additional columns for day/time and updated toll rates.
    """
    # Define days and time ranges
    days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    time_ranges = [
        (time(0, 0), time(10, 0)),    # 00:00:00 to 10:00:00
        (time(10, 0), time(18, 0)),   # 10:00:00 to 18:00:00
        (time(18, 0), time(23, 59, 59))  # 18:00:00 to 23:59:59
    ]
    
    # Discount factors
    weekday_discounts = [0.8, 1.2, 0.8]  # for respective time ranges
    weekend_discount = 0.7  # for weekends
    
    # Create an empty list to store rows of the new DataFrame
    new_rows = []

    # Iterate over each id_start, id_end pair
    for _, row in df.iterrows():
        id_start = row['id_start']
        id_end = row['id_end']
        distance = row['distance']
        
        # Create time-based entries for each day
        for day in days:
            for time_range in time_ranges:
                start_time, end_time = time_range

                # Calculate toll rates based on time
                if day in ['Saturday', 'Sunday']:
                    discount = weekend_discount
                else:
                    # Determine which discount to apply based on the time range
                    if start_time == time(0, 0):
                        discount = weekday_discounts[0]
                    elif start_time == time(10, 0):
                        discount = weekday_discounts[1]
                    else:
                        discount = weekday_discounts[2]

                # Calculate the toll rates
                moto_rate = distance * 0.8 * discount
                car_rate = distance * 1.2 * discount
                rv_rate = distance * 1.5 * discount
                bus_rate = distance * 2.2 * discount
                truck_rate = distance * 3.6 * discount

                # Append the new row
                new_rows.append({
                    'id_start': id_start,
                    'id_end': id_end,
                    'distance': distance,
                    'start_day': day,
                    'start_time': start_time,
                    'end_day': day,
                    'end_time': end_time,
                    'moto': moto_rate,
                    'car': car_rate,
                    'rv': rv_rate,
                    'bus': bus_rate,
                    'truck': truck_rate
                })

    # Create the new DataFrame from the collected rows
    new_df = pd.DataFrame(new_rows)

    return new_df

# Example usage
toll_rate_df = pd.DataFrame({
    'id_start': [1001400, 1001400, 1001400],
    'id_end': [1001402, 1001404, 1001406],
    'distance': [9.7, 29.9, 45.9],
    'moto': [7.76, 23.92, 36.72],
    'car': [11.64, 35.88, 55.08],
    'rv': [14.55, 44.85, 68.85],
    'bus': [21.34, 65.78, 100.98],
    'truck': [34.92, 107.64, 165.24]
})

# Call the function to calculate time-based toll rates
time_based_toll_df = calculate_time_based_toll_rates(toll_rate_df)

# Print the first 9 rows of the resulting DataFrame
print(time_based_toll_df.head(9))


    id_start     id_end  distance  start_day start_time    end_day  end_time  \
0  1001400.0  1001402.0       9.7     Monday   00:00:00     Monday  10:00:00   
1  1001400.0  1001402.0       9.7     Monday   10:00:00     Monday  18:00:00   
2  1001400.0  1001402.0       9.7     Monday   18:00:00     Monday  23:59:59   
3  1001400.0  1001402.0       9.7    Tuesday   00:00:00    Tuesday  10:00:00   
4  1001400.0  1001402.0       9.7    Tuesday   10:00:00    Tuesday  18:00:00   
5  1001400.0  1001402.0       9.7    Tuesday   18:00:00    Tuesday  23:59:59   
6  1001400.0  1001402.0       9.7  Wednesday   00:00:00  Wednesday  10:00:00   
7  1001400.0  1001402.0       9.7  Wednesday   10:00:00  Wednesday  18:00:00   
8  1001400.0  1001402.0       9.7  Wednesday   18:00:00  Wednesday  23:59:59   

    moto     car     rv     bus   truck  
0  6.208   9.312  11.64  17.072  27.936  
1  9.312  13.968  17.46  25.608  41.904  
2  6.208   9.312  11.64  17.072  27.936  
3  6.208   9.312  11.64  17.072